In [1]:
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
import random


In [2]:
if torch.cuda.is_available():
    DTYPE = torch.cuda.FloatTensor
else:
    DTYPE = torch.FloatTensor
    

In [3]:
data = open('texts/Lovecraft.txt','r').read()
alphabet = set(data)

ix_to_char = {k:v for k,v in enumerate(alphabet)}
char_to_ix = {k:v for v,k in enumerate(alphabet)}

In [4]:
NUM_LAYERS = 1
BATCH_SIZE = 1
HIDDEN_DIM = 64
SEQ_LEN = 65


In [5]:
def sequence_to_tensor(sequence):
    tensor = torch.zeros(SEQ_LEN,len(alphabet)).type(DTYPE)
    for i, c in enumerate(sequence):
        tensor[i][char_to_ix[c]] = 1
    return tensor


In [15]:
class LSTM(nn.Module):

    def __init__(self,alphabet_size, hidden_dim, output_size):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.i2h = nn.Linear(alphabet_size,hidden_dim)
        self.lstm = nn.LSTM(hidden_dim,hidden_dim,NUM_LAYERS)
        self.h2O = nn.Linear(hidden_dim, output_size)
        
        self.hidden = self.init_hidden()
        
        
    def init_hidden(self):
       
        return (autograd.Variable(torch.zeros(NUM_LAYERS, BATCH_SIZE, self.hidden_dim).type(DTYPE)),
                autograd.Variable(torch.zeros(NUM_LAYERS, BATCH_SIZE, self.hidden_dim).type(DTYPE)))

    def forward(self, sequence):
        out = self.i2h(sequence)
        lstm_out, self.hidden = self.lstm(out.view(SEQ_LEN-1,BATCH_SIZE,-1),self.hidden)
        out = self.h2O(lstm_out.view(-1,self.hidden_dim))
        return out
    
    
    def gen_text(self, sequence,t=None):
    
        inputs = autograd.Variable(sequence_to_tensor(sequence))
        idxs = torch.zeros(inputs.data.size())
        out = self(inputs[:-1,:])

        if t != None:
            soft_out = F.softmax(out/t,dim=1)
            
            for i in range(soft_out.size()[0]):
                idxs[i] = np.random.choice(soft_out.size()[1],p=soft_out.data.numpy()[i])
                
        else:
            idxs = out.max(1)[1].data

        
        return out,idxs
    


In [14]:
rnn = LSTM(len(alphabet),HIDDEN_DIM,len(alphabet)).type(DTYPE)
optimizer = optim.Adam(rnn.parameters(),lr=0.01)
criterion = nn.CrossEntropyLoss()

epochs = 100

In [13]:
def train(data):
    
    rnn.train(True)
    for epoch in range(epochs):

        losses = np.array([])
        
        for i in range(0,len(data),SEQ_LEN):

            rnn.zero_grad()
            rnn.hidden = rnn.init_hidden()
            
            inputs = autograd.Variable(sequence_to_tensor(data[i:i + SEQ_LEN]))
                        
            out = rnn(inputs[:-1,:])
                        
            _,target = inputs[1:,:].topk(1)
            
            loss = criterion(out.view(-1,len(alphabet)),target.view(-1))
            losses = np.append(losses,loss.data[0])
            
            loss.backward()
            optimizer.step()


        print("Epoch {}/{}\nLoss: {:.2f}".format(epoch+1,epochs,losses.mean()))
        print("="*15)


In [17]:
train(data[0:10000])

Epoch 1/100
Loss: 2.96
Epoch 2/100
Loss: 2.48
Epoch 3/100
Loss: 2.32
Epoch 4/100
Loss: 2.23
Epoch 5/100
Loss: 2.15
Epoch 6/100
Loss: 2.08
Epoch 7/100
Loss: 2.01
Epoch 8/100
Loss: 1.96
Epoch 9/100
Loss: 1.91
Epoch 10/100
Loss: 1.87
Epoch 11/100
Loss: 1.83
Epoch 12/100
Loss: 1.79
Epoch 13/100
Loss: 1.76
Epoch 14/100
Loss: 1.73
Epoch 15/100
Loss: 1.69
Epoch 16/100
Loss: 1.66
Epoch 17/100
Loss: 1.64
Epoch 18/100
Loss: 1.62
Epoch 19/100
Loss: 1.58
Epoch 20/100
Loss: 1.57
Epoch 21/100
Loss: 1.54
Epoch 22/100
Loss: 1.53
Epoch 23/100
Loss: 1.53
Epoch 24/100
Loss: 1.51
Epoch 25/100
Loss: 1.48
Epoch 26/100
Loss: 1.47
Epoch 27/100
Loss: 1.46
Epoch 28/100
Loss: 1.45
Epoch 29/100
Loss: 1.45
Epoch 30/100
Loss: 1.41
Epoch 31/100
Loss: 1.40
Epoch 32/100
Loss: 1.42
Epoch 33/100
Loss: 1.42
Epoch 34/100
Loss: 1.38
Epoch 35/100
Loss: 1.39
Epoch 36/100
Loss: 1.38
Epoch 37/100
Loss: 1.36
Epoch 38/100
Loss: 1.35
Epoch 39/100
Loss: 1.33
Epoch 40/100
Loss: 1.31
Epoch 41/100
Loss: 1.30
Epoch 42/100
Loss: 1.31
E

In [21]:
string = ''  


rnn.train(False)

_ ,idxs = rnn.gen_text(data[0:0+SEQ_LEN])
    
#for i in range(100):
    
    #out = rnn(out)
    #soft_out = F.softmax(out/t,dim=1)

for c in idxs:
    string += ix_to_char[c]

print(string)           



#print(string,file=open('texts/output.txt','w'))
    
    #with open('texts/output.txt', mode='wt', encoding='utf-8') as myfile:
        #myfile.writelines(' '.join(lyrics))


  rworror.w  tliy

The sost se ciagl Whasx wn the sanld, w shink


In [ ]:
# install cudnn
# overwrite dataloader
# encapsulate last cell in function for text generation
#use temperature only when t != none